# GUI Basics with Panel:

In [1]:
import panel as pn
pn.extension()

slider = pn.widgets.IntSlider(name='My Slider', start=0, end=100, value=50)
text_input = pn.widgets.TextInput(name='Your Name', placeholder='Type here...')
button = pn.widgets.Button(name='Click me')

# Display widgets in a column
pn.Column(slider, text_input, button)

Column
    [0] IntSlider(end=100, name='My Slider', value=50)
    [1] TextInput(name='Your Name', placeholder='Type here...')
    [2] Button(name='Click me')

In [2]:
import panel as pn
import matplotlib.pyplot as plt
import numpy as np
import io
import base64

pn.extension('matplotlib')

# Define parameter names (editable by YOU, not user)
# Can add in experimental settings later
setting_names = ["MAGTYPE", "R0MM", "LAYERS", "RFEMM", "RBENDMM", "NBEND", "VC2CB", "VC2CE"]
layer_names = ["Straight Num Blocks", "End Num Blocks", "Straight Length", "Wire Diam", "Layer Coil Rad", "Wire Current", "End Wire Insul", "Straight Wire Insul"]

# Create text input widgets
setting_inputs = [pn.widgets.TextInput(name=name, placeholder="Enter value") for name in setting_names]
layer_inputs = [pn.widgets.TextInput(name=name, placeholder="Enter description") for name in layer_names]

# Group into columns
settings_column = pn.Column(pn.pane.Markdown("## Settings"), *setting_inputs)
layers_column = pn.Column(pn.pane.Markdown("## Layer Descriptions"), *layer_inputs)

# Plotting output pane
plot_pane = pn.pane.Matplotlib(sizing_mode='stretch_width', height=400)

# Button callback
def plot_radius_curve(event):
    try:
        # Assume the first input is "radius" values, comma-separated
        radius_input = setting_inputs[1].value.strip()
        if not radius_input:
            plot_pane.object = None
            return

        # Convert to float array
        radii = np.array([float(r.strip()) for r in radius_input.split(',')])
        theta = np.linspace(0, 2*np.pi, len(radii))

        # Plot in polar
        fig, ax = plt.subplots(subplot_kw={'polar': True})
        ax.plot(theta, radii, marker='o', color='blue')
        ax.set_title("Polar Radius Plot")

        # Update pane
        plot_pane.object = fig

    except Exception as e:
        print("Plot error:", e)
        plot_pane.object = None

# Plot button
plot_button = pn.widgets.Button(name="Plot Radius Curve", button_type='primary')
plot_button.on_click(plot_radius_curve)

# Layout
layout = pn.Column(
    pn.Row(settings_column, pn.Spacer(width=50), layers_column),
    pn.Spacer(height=20),
    plot_button,
    plot_pane,
    sizing_mode='stretch_width'
)

# --- Help Button ---
help_button = pn.widgets.Button(name="Help", button_type="default")

# --- Help View ---
back_button = pn.widgets.Button(name="← Back", button_type="primary")

help_text = """
# Config File Definitions:

## Settings
* VC2CB=True -> Flag to allow variable conductor spacing in body
* VC2CE=True -> Flag to allow variable conductor spacing in ends
* MAGTYPE=2 -> 2 for Dipole; 4 for quadrupole; 3 for combine function
* LAYERS=2 -> Total Number of Layers
* RFEMM=110 -> Iron Inner Radius in mm; If zero, No Iron
* R0MM=40 -> Reference Radius in mm
* RBENDMM=10 -> Bend Radius at Transition to End Sector
* NBEND=10 -> Number of Points on Bend Curve

## Layer Descriptions
* 1 -> Number of Blocks in Layer Straight Section
* 2 -> Number of Blocks in Layer End Section
* 3 -> Straight Section Length (m)
* 4 -> Wire Diameter (mm)
* 5 -> Layer Coil Radius (mm)
* 6 -> Wire Current (A)
* 7 -> Wire Insulation in End Section (mm)
* 8 -> Wire Insulation in Straight Section (mm)

"""

help_view = pn.Column(
    back_button,
    pn.pane.Markdown(help_text, sizing_mode='stretch_width'),
    visible=False  # Initially hidden
)

# --- Main View ---
main_view = pn.Column(
    pn.Row(
        pn.Column(pn.pane.Markdown("## Settings"), *setting_inputs),
        pn.Spacer(width=50),
        pn.Column(pn.pane.Markdown("## Layer Descriptions"), *layer_inputs),
    ),
    pn.Spacer(height=15),
    pn.Row(plot_button, pn.Spacer(width=10), help_button),
    plot_pane,
    visible=True  # Initially shown
)

# --- View Swapping Logic ---
def show_help(event):
    main_view.visible = False
    help_view.visible = True

def show_main(event):
    help_view.visible = False
    main_view.visible = True

help_button.on_click(show_help)
back_button.on_click(show_main)

# --- Root layout ---
layout = pn.Column(main_view, help_view, sizing_mode='stretch_width')
layout.servable()


Column(sizing_mode='stretch_width')
    [0] Column
        [0] Row
            [0] Column
                [0] Markdown(str)
                [1] TextInput(name='MAGTYPE', placeholder='Enter value')
                [2] TextInput(name='R0MM', placeholder='Enter value')
                [3] TextInput(name='LAYERS', placeholder='Enter value')
                [4] TextInput(name='RFEMM', placeholder='Enter value')
                [5] TextInput(name='RBENDMM', placeholder='Enter value')
                [6] TextInput(name='NBEND', placeholder='Enter value')
                [7] TextInput(name='VC2CB', placeholder='Enter value')
                [8] TextInput(name='VC2CE', placeholder='Enter value')
            [1] Spacer(width=50)
            [2] Column
                [0] Markdown(str)
                [1] TextInput(name='Straight Num Blocks', placeholder='Enter description')
                [2] TextInput(name='End Num Blocks', placeholder='Enter description')
                [3] TextInput(name='Straight Length', placeholder='Enter description')
                [4] TextInput(name='Wire Diam', placeholder='Enter description')
                [5] TextInput(name='Layer Coil Rad', placeholder='Enter description')
                [6] TextInput(name='Wire Current', placeholder='Enter description')
                [7] TextInput(name='End Wire Insul', placeholder='Enter description')
                [8] TextInput(name='Straight Wire Insul', placeholder='Enter description')
        [1] Spacer(height=15)
        [2] Row
            [0] Button(button_type='primary', name='Plot Radius Curve')
            [1] Spacer(width=10)
            [2] Button(name='Help')
        [3] Matplotlib(None, height=400, sizing_mode='stretch_width')
    [1] Column(visible=False)
        [0] Button(button_type='primary', name='← Back')
        [1] Markdown(str, sizing_mode='stretch_width')

In [3]:
import panel as pn
import matplotlib.pyplot as plt
import numpy as np

pn.extension('matplotlib')

# ----------------------------
# CONFIG
# ----------------------------
setting_names = ["MAGTYPE", "R0MM", "LAYERS", "RFEMM", "RBENDMM", "NBEND", "VC2CB", "VC2CE"]
layer_names = ["Straight Num Blocks", "End Num Blocks", "Straight Length", "Wire Diam",
               "Layer Coil Rad", "Wire Current", "End Wire Insul", "Straight Wire Insul"]

# ----------------------------
# WIDGETS
# ----------------------------
# Settings input fields
setting_inputs = [pn.widgets.TextInput(name=name, placeholder="Enter value") for name in setting_names]
settings_column = pn.Column(pn.pane.Markdown("## Settings"), *setting_inputs)

# Layer control
layer_count_input = pn.widgets.IntInput(name="How many layers?", value=1, start=1, step=1)
generate_layers_button = pn.widgets.Button(name="Generate Layer Settings", button_type="success")
layer_control_row = pn.Row(layer_count_input, generate_layers_button)

# Dynamic layer settings container
dynamic_layers_column = pn.Row()

# ----------------------------
# LAYER GENERATOR CALLBACK
# ----------------------------
def generate_layer_settings(event):
    dynamic_layers_column.clear()
    num_layers = layer_count_input.value

    for i in range(1, num_layers + 1):
        title = pn.pane.Markdown(f"## Layer {i}")
        inputs = [pn.widgets.TextInput(name=name, placeholder=f"Layer {i} - {name}") for name in layer_names]
        layer_section = pn.Column(title, *inputs, sizing_mode='stretch_width')
        dynamic_layers_column.append(layer_section)

generate_layers_button.on_click(generate_layer_settings)

# ----------------------------
# PLOTTING
# ----------------------------
plot_pane = pn.pane.Matplotlib(sizing_mode='stretch_width', height=400)

def plot_radius_curve(event):
    try:
        radius_input = setting_inputs[1].value.strip()  # Assume R0MM is radius
        if not radius_input:
            plot_pane.object = None
            return

        radii = np.array([float(r.strip()) for r in radius_input.split(',')])
        theta = np.linspace(0, 2 * np.pi, len(radii))

        fig, ax = plt.subplots(subplot_kw={'polar': True})
        ax.plot(theta, radii, marker='o', color='blue')
        ax.set_title("Polar Radius Plot")
        plot_pane.object = fig

    except Exception as e:
        print("Plot error:", e)
        plot_pane.object = None

plot_button = pn.widgets.Button(name="Plot Radius Curve", button_type='primary')
plot_button.on_click(plot_radius_curve)

# ----------------------------
# HELP PANEL
# ----------------------------
help_button = pn.widgets.Button(name="Help", button_type="default")
back_button = pn.widgets.Button(name="← Back", button_type="primary")

help_text = """
# Config File Definitions:

## Settings
* VC2CB=True → Variable conductor spacing in body
* VC2CE=True → Variable conductor spacing in ends
* MAGTYPE=2 → 2 for Dipole; 4 for quadrupole; 3 for combine function
* LAYERS=2 → Total Number of Layers
* RFEMM=110 → Iron Inner Radius in mm (0 = No Iron)
* R0MM=40 → Reference Radius in mm
* RBENDMM=10 → Bend Radius at Transition to End Sector
* NBEND=10 → Number of Points on Bend Curve

## Layer Descriptions
1. Straight Section: Number of Blocks
2. End Section: Number of Blocks
3. Straight Section Length (m)
4. Wire Diameter (mm)
5. Layer Coil Radius (mm)
6. Wire Current (A)
7. End Section Wire Insulation (mm)
8. Straight Section Wire Insulation (mm)
"""

help_view = pn.Column(
    back_button,
    pn.pane.Markdown(help_text, sizing_mode='stretch_width'),
    visible=False
)

# ----------------------------
# MAIN PANEL
# ----------------------------
main_view = pn.Column(
    pn.Row(settings_column, pn.Spacer(width=25), dynamic_layers_column),
    pn.Spacer(height=15),
    layer_control_row,
    pn.Spacer(height=15),
    pn.Row(plot_button, pn.Spacer(width=10), help_button),
    pn.Spacer(height=15),
    plot_pane,
    visible=True
)

# ----------------------------
# VIEW SWITCHING
# ----------------------------
def show_help(event):
    main_view.visible = False
    help_view.visible = True

def show_main(event):
    help_view.visible = False
    main_view.visible = True

help_button.on_click(show_help)
back_button.on_click(show_main)

# ----------------------------
# ROOT LAYOUT
# ----------------------------
layout = pn.Column(main_view, help_view, sizing_mode='stretch_width')
layout.servable()


Column(sizing_mode='stretch_width')
    [0] Column
        [0] Row
            [0] Column
                [0] Markdown(str)
                [1] TextInput(name='MAGTYPE', placeholder='Enter value')
                [2] TextInput(name='R0MM', placeholder='Enter value')
                [3] TextInput(name='LAYERS', placeholder='Enter value')
                [4] TextInput(name='RFEMM', placeholder='Enter value')
                [5] TextInput(name='RBENDMM', placeholder='Enter value')
                [6] TextInput(name='NBEND', placeholder='Enter value')
                [7] TextInput(name='VC2CB', placeholder='Enter value')
                [8] TextInput(name='VC2CE', placeholder='Enter value')
            [1] Spacer(width=25)
            [2] Row()
        [1] Spacer(height=15)
        [2] Row
            [0] IntInput(name='How many layers?', start=1, value=1)
            [1] Button(button_type='success', name='Generate Layer Settings')
        [3] Spacer(height=15)
        [4] Row
            [0] Button(button_type='primary', name='Plot Radius Curve')
            [1] Spacer(width=10)
            [2] Button(name='Help')
        [5] Spacer(height=15)
        [6] Matplotlib(None, height=400, sizing_mode='stretch_width')
    [1] Column(visible=False)
        [0] Button(button_type='primary', name='← Back')
        [1] Markdown(str, sizing_mode='stretch_width')